In [ ]:
# Cell 1: Installation
!pip install owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 57.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.49-py3-none-any.whl size=23742426 sha256=3eecd8cd29cd6dfb7871450e0de072d7a139e99a956baf9b26ac99c4905955fc
  Stored in directory: /root/.cache/pip/wheels/43/fe/dc/a0de3c289cfd5923ece6524469d328950e14fa0c90b1088ffa
Successfully built owlready2


In [2]:
from owlready2 import *
import json
import re


# We define namespaces
dul_onto = get_ontology("http://www.ease-crc.org/ont/DUL.owl").load()
dul_ns = get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
onto = get_ontology("http://www.semanticweb.org/gamematcher.owl")
onto.imported_ontologies.append(dul_onto)

with onto:
    onto.metadata.comment.append("Andreea Scrob, Edoardo Tommasi")
    onto.metadata.comment.append("GameMatcher is an intelligent companion system that assists users in finding video games compatible with their computer.")
    onto.metadata.label.append("GameMatcher Ontology")

    # TAXONOMY (CLASSES)
    #  USER
    class User(Thing):
        comment = ["An Agent who interacts with the system to find compatible video games."]
        label = ["User"]
        is_a = [dul_ns.Agent]

    # COMPUTER and COMPONENTS
    # Note: A Computer is a Physical Object, not a subclass of Component.
    class Computer(Thing):
        comment = ["A Physical Object consisting of hardware components capable of executing software."]
        label = ["Computer"]
        is_a = [dul_ns.PhysicalObject]

    class Component(Thing):
        comment = ["A Physical Object that forms a constituent part of a computing system."]
        label = ["Component"]
        is_a = [dul_ns.PhysicalObject]

    # Specific Hardware Classes
    class CPU(Component):
        comment = ["A Hardware Component that acts as the primary processing unit."]
        label = ["CPU"]

    class GPU(Component):
        comment = ["A Hardware Component specialized for rendering graphics."]
        label = ["GPU"]

    class RAM(Component):
        comment = ["A Hardware Component used for temporary data storage."]
        label = ["RAM"]

    class Storage(Component):#da specificare nella relazione che questo diventa Storage poi lo spazio disponibile viene preso con la dataproperty
        comment = ["A Hardware Component used for data retention (e.g., HDD or SSD)."]
        label = ["Storage"]

    # Specific Software Classes
    class OperatingSystem(Thing):
        comment = ["A Software Component that manages computer hardware and resources."]
        label = ["Operating System"]
        is_a = [dul_ns.InformationObject]

    #  VIDEOGAME and REQUIREMENTS
    class VideoGame(Thing):
        comment = ["An Information Object representing a digital game available for play."]
        label = ["Video Game"]
        is_a = [dul_ns.InformationObject]

    class MinimumRequirement(Thing):
        comment = ["A Requirement specifying the absolute minimum specifications for a game to run."]
        label = ["Minimum Requirement"]
        is_a = [dul_ns.Description]


    #  ATTRIBUTES and CONCEPTS
    #todo: chiedere al prof se possiamo aggiungere GameAttribute
    class GameAttribute(Thing):
        comment = ["A Concept representing a characteristic feature of a video game."]
        label = ["Game Attribute"]
        is_a = [dul_ns.Concept]

    class GameGenre(GameAttribute):
        comment = ["A classification based on gameplay interaction (e.g., RPG, Strategy)."]
        label = ["Game Genre"]

    class PlayerMode(GameAttribute):
        comment = ["Defines the number of players supported (e.g., Single-player)."]
        label = ["Player Mode"]

    class PEGIRating(GameAttribute):
        comment = ["Indicates the suitability of content for different age groups."]
        label = ["PEGI Rating"]

    # 1. Top-Level Disjointness
    # A User is not a Computer, which is not a Game, etc.
    AllDisjoint([User, Computer, Component, OperatingSystem, VideoGame, MinimumRequirement, GameAttribute])

    # 2. Hardware Component Disjointness
    # A CPU is not a GPU, RAM, or Storage.
    AllDisjoint([CPU, GPU, RAM, Storage])

    # 3. Attribute Disjointness
    # A Genre is not a PEGI Rating or a Player Mode.
    AllDisjoint([GameGenre, PlayerMode, PEGIRating])


    # OBJECT PROPERTIES (RELATIONS)
    # Relations: Computer -> Components
    class hasComponent(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]
        range = [Component]
        comment = ["Generic relation linking a computer to its parts."]
        label = ["has component"]


    # Sub-properties for specificity
    class hasCPU(hasComponent, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [CPU]
        comment = ["Relation linking components to the computer's cpu."]
        label = ["has CPU"]

    class hasGPU(hasComponent, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [GPU]
        comment = ["Relation linking components to the computer's gpu."]
        label = ["has GPU"]

    class hasRAM(hasComponent, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [RAM]
        comment = ["Relation linking components to the computer's ram."]
        label = ["has RAM"]

    class hasStorage(hasComponent, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [Storage]
        comment = ["Relation linking components to the computer's  available storage."]
        label = ["has storage"]

    class hasOS(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [OperatingSystem]
        comment = ["Relation linking components to the computer's os."]
        label = ["has operating system"]


    # Relations: Videogame -> Requirements
    class hasMinRequirement(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]
        range = [MinimumRequirement]
        comment = ["Relation linking videogame to its component requirements to make it run."]
        label = ["has minimum requirement"]


    # Relations: User -> Computer
    class hasComputer(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]
        range = [Computer]
        comment = ["Relation linking user to the computer he owns."]
        label = ["has computer device"]



    # Relations: Attribute
    class hasGenre(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [GameGenre]
        comment = ["Relation linking videogame to its genre."]
        label = ["has genre"]


    class preferredGenre(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]; range = [GameGenre]
        comment = ["Relation linking user to its preferred game's genre."]
        label = ["prefers genre"]

    class hasPlayerMode(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [PlayerMode]
        comment = ["Relation linking videogame to its player mode."]
        label = ["has player mode"]


    class preferredPlayerMode(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]; range = [PlayerMode]
        comment = ["Relation linking user to its preferred game mode."]
        label = ["prefers player mode"]


    class hasPEGI(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [PEGIRating]
        comment = ["Relation linking videogame to its age rating."]
        label = ["has PEGI rating"]


    # Derived Relation (for Compatibility Checking)
    class isCompatibleWith(ObjectProperty, SymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]
        range = [Computer]
        comment = ["Inferred relation: true if the computer meets all game requirements."]
        label = ["is compatible with"]



    # DATA PROPERTIES (ACTIONABLE LOGIC)

    # BENCHMARK SCORE (CPU and GPU)
    # User Hardware Property
    class hasBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [CPU, GPU]
        range = [int]
        comment = ["The performance score of the user's hardware (e.g. PassMark)."]
        label = ["has benchmark score"]


    # Game Requirement Property
    class requiresBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [int]
        comment = ["The minimum benchmark score required by the game."]
        label = ["requires benchmark score"]


    #  MEMORY (RAM and VRAM)
    class hasMemorySizeGB(DataProperty, FunctionalProperty):
        domain = [RAM, GPU]
        range = [float]
        comment = ["The amount of memory in Gigabytes."]
        label = ["has memory size (GB)"]


    class requiresMemoryGB(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Minimum RAM or VRAM required in Gigabytes."]
        label = ["requires memory (GB)"]


    #  STORAGE
    class hasStorageSizeGB(DataProperty, FunctionalProperty):
        domain = [Storage]
        range = [float]
        comment = ["Total available disk space in Gigabytes."]
        label = ["has storage size (GB)"]


    class requiresStorageSpaceGB(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Installation size required in Gigabytes."]
        label = ["requires storage space (GB)"]


    # OS LOGIC
    class hasOSVersionValue(DataProperty, FunctionalProperty):
        domain = [OperatingSystem]
        range = [float]
        comment = ["Numeric version of the OS (e.g., 10.0, 11.0)."]
        label = ["has OS version value"]


    class requiresMinOSVersionValue(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Minimum numeric OS version required."]
        label = ["requires min OS version"]



   #  BUDGET and PRICING
    class hasPrice(DataProperty, FunctionalProperty):
        domain = [VideoGame]
        range = [float]
        comment = ["The monetary cost required to purchase the digital game."]
        label = ["has price"]


    class hasBudget(DataProperty, FunctionalProperty):
        domain = [User]
        range = [float]
        comment = ["The maximum monetary limit the user is willing to spend."]
        label = ["has budget"]


    #  AGE and PEGI
    class hasAge(DataProperty, FunctionalProperty):
        domain = [User]
        range = [int]
        comment = ["The chronological age of the user (in years), used for PEGI checks."]
        label = ["has age"]


    class hasPEGIAgeThreshold(DataProperty, FunctionalProperty):
        domain = [PEGIRating]
        range = [int]
        comment = ["The numeric age threshold (e.g., 18)."]
        label = ["has PEGI threshold"]




In [4]:
# ==========================================
# TBOX REFINEMENTS (VINCOLI E REGOLE)
# ==========================================
# ATTENZIONE: Queste righe devono essere SENZA spazi all'inizio!

# 1. DISJOINTNESS
AllDisjoint([User, Computer, Component, OperatingSystem, VideoGame, MinimumRequirement, GameAttribute])
AllDisjoint([CPU, GPU, RAM, Storage])
AllDisjoint([GameGenre, PlayerMode, PEGIRating])

# 2. RESTRICTIONS
# Computer
Computer.is_a.append(hasCPU.exactly(1, CPU))
Computer.is_a.append(hasGPU.exactly(1, GPU))
Computer.is_a.append(hasRAM.min(1, RAM))
Computer.is_a.append(hasStorage.min(1, Storage))
Computer.is_a.append(hasOS.exactly(1, OperatingSystem))

# VideoGame
VideoGame.is_a.append(hasPrice.exactly(1, float))
VideoGame.is_a.append(hasPEGI.exactly(1, PEGIRating))
VideoGame.is_a.append(hasMinRequirement.min(1, MinimumRequirement))

# User
User.is_a.append(hasAge.exactly(1, int))
User.is_a.append(hasBudget.exactly(1, float))

print("Regole e Restrizioni applicate con successo!")

Regole e Restrizioni applicate con successo!


In [ ]:

# FUNZIONI DI UTILITÀ (Parsing & Matching)

def clean_iri(text):
    """Rimuove caratteri illegali per creare un IRI valido."""
    if not text: return "Unknown"
    # Sostituisce spazi e caratteri strani con underscore
    clean = re.sub(r'[^a-zA-Z0-9]', '_', str(text))
    # Rimuove underscore multipli
    clean = re.sub(r'_+', '_', clean)
    return clean.strip('_')

def parse_gb(text):
    """Estrae i GB da stringhe come '8 GB RAM' o '100 GB available'."""
    if not text: return 0.0
    match = re.search(r'(\d+)', str(text))
    if match:
        return float(match.group(1))
    return 0.0

def parse_price(text):
    """Estrae il prezzo da stringhe come '39,99€' o 'Free To Play'."""
    if not text: return 0.0
    if "Free" in text: return 0.0
    # Sostituisce virgola con punto ed estrae numeri
    text = text.replace(',', '.')
    match = re.search(r'(\d+\.\d+)', text)
    if match:
        return float(match.group(1))
    return 0.0

def find_hardware_score(req_text, hardware_db):
    """
    Cerca il punteggio benchmark.
    req_text: Stringa da Steam (es. 'NVIDIA GeForce GTX 1060')
    hardware_db: Lista di dizionari dal JSON benchmark
    """
    if not req_text: return None
    
    req_clean = req_text.lower().replace('-', ' ').replace('_', ' ')
    
    best_match = None
    best_score = 0
    
    # Logica semplice: se il nome del benchmark è contenuto nel requisito
    for item in hardware_db:
        bench_name = item['name'].lower().replace('-', ' ')
        # Esempio: "geforce gtx 1060" in "nvidia geforce gtx 1060" -> VERO
        if bench_name in req_clean:
            # Troviamo il match più lungo (solitamente più preciso)
            if best_match is None or len(bench_name) > len(best_match):
                best_match = bench_name
                best_score = int(item['mark'])
    
    return best_score

In [ ]:
try:
    with open('/home/andreea/GameMatcher/D3/Scraping/cleaning/filtered_cpu_data.json', 'r') as f: cpu_data = json.load(f)
    with open('/home/andreea/GameMatcher/D3/Scraping/cleaning/filtered_gpu_data.json', 'r') as f: gpu_data = json.load(f)
    with open('/home/andreea/GameMatcher/D3/Scraping/cleaning/steam_parsed_CLEAN.json', 'r') as f: game_data = json.load(f)
    print("File JSON caricati correttamente.")
except FileNotFoundError:
    print("Errore: Assicurati di aver caricato i file JSON nella directory corrente.")
    cpu_data, gpu_data, game_data = [], [], []

with onto:
    
 
    print("Popolamento Hardware...")
    # Mappa per accesso veloce dopo
    cpu_db = cpu_data # Lo usiamo per il matching
    gpu_db = gpu_data
    
    # Creiamo istanze CPU nel grafo
    for cpu in cpu_data:
        cpu_iri = clean_iri(cpu['name'])
        new_cpu = onto.CPU(f"CPU_{cpu_iri}")
        new_cpu.label = [cpu['name']]
        new_cpu.hasBenchmarkScore = int(cpu['mark'])
        
    # Creiamo istanze GPU nel grafo
    for gpu in gpu_data:
        gpu_iri = clean_iri(gpu['name'])
        new_gpu = onto.GPU(f"GPU_{gpu_iri}")
        new_gpu.label = [gpu['name']]
        new_gpu.hasBenchmarkScore = int(gpu['mark'])

  
    print("Popolamento Giochi...")
    for game in game_data:
        # 1. Dati base gioco
        game_name = game.get('NAME', 'UnknownGame')
        game_iri = clean_iri(game_name)
        new_game = onto.VideoGame(f"Game_{game_iri}")
        new_game.label = [game_name]
        
        # Prezzo
        price_val = parse_price(game.get('PRICE'))
        new_game.hasPrice = price_val
        
        # PEGI (Gestione Rating)
        pegi_text = game.get('PEGI', 'N/A')
        if pegi_text and pegi_text != 'N/A' and not pegi_text.startswith('#'):
            pegi_iri = clean_iri(pegi_text)
            # Creiamo l'istanza PEGI se non esiste
            pegi_instance = onto.PEGIRating(f"PEGI_{pegi_iri}")
            pegi_instance.label = [pegi_text]
            # Estrazione soglia numerica (es. 18 da "18+")
            age_match = re.search(r'\d+', pegi_text)
            if age_match:
                pegi_instance.hasPEGIAgeThreshold = int(age_match.group(0))
            new_game.hasPEGI = pegi_instance

        # MODALITÀ DI GIOCO (Player Mode)
        mode_text = game.get('MODE', '')
        if mode_text:
            # A volte ci sono più modalità separate da virgola
            modes = [m.strip() for m in mode_text.split(',')]
            for m in modes:
                mode_iri = clean_iri(m)
                mode_instance = onto.PlayerMode(f"Mode_{mode_iri}")
                mode_instance.label = [m]
                new_game.hasPlayerMode.append(mode_instance)

        # 2. Requisiti Minimi
        req_iri = f"Req_{game_iri}"
        new_req = onto.MinimumRequirement(req_iri)
        new_game.hasMinRequirement = new_req
        
        # RAM e Storage
        new_req.requiresMemoryGB = parse_gb(game.get('RAM'))
        new_req.requiresStorageSpaceGB = parse_gb(game.get('STORAGE'))
        
        # --- MATCHING HARDWARE ---
        # Cerchiamo il punteggio GPU
        # Proviamo le colonne GPU_0, GPU_1... prendiamo la prima valida
        required_gpu_score = 0
        for k in ['GPU_0', 'GPU_1', 'GPU_2']:
            val = game.get(k)
            score = find_hardware_score(val, gpu_db)
            if score:
                required_gpu_score = score
                # Opzionale: aggiungi label per debug
                new_req.label.append(f"Requires GPU akin to {val}")
                break # Trovato un requisito valido
        
        if required_gpu_score > 0:
            new_req.requiresBenchmarkScore = required_gpu_score
            
        # Nota: Se vuoi fare anche la CPU, puoi replicare la logica sopra
        # usando cpu_db e le colonne CPU_0, CPU_1...
        # Per ora il tuo modello ha "requiresBenchmarkScore" generico, 
        # potresti voler distinguere requiresCPUScore e requiresGPUScore 
        # modificando la TBox, oppure usare lo score GPU come collo di bottiglia principale.

print("ABox creata con successo!")


output_file = "gameMatcher_ABox.owl"
onto.save(file=output_file)
print(f"Ontologia salvata in: {output_file}")

File JSON caricati correttamente.
Popolamento Hardware...
Popolamento Giochi...
ABox creata con successo!
Ontologia salvata in: gameMatcher_ABox.owl
